# El Lab (DAA Project)

En un laboratorio de la BIOFAM quieren modificar genéticamente una gallina para que sea más resistente a las enfermedades. Para esto deben de cambiar la secuencia de aminoácidos presentes en su genoma. La secuencia que quieren reproducir es la de los cocodrilos, que se sabe que son animales que han existido por millones de años.  
Entonces, dado que una secuencia de aminoácidos se puede codificar como una secuencia de caracteres se tiene:  

- $T$ es la secuencia de aminoácidos de los cocodrilos (tamaño m)  
- $S$ es la secuencia de aminoácidos de la gallina original (tamaño n)
- $A$ es la secuencia de aminoácidos de la galllina a crear (inicialmente vacía)

Para lograr que la gallina modificada adquiera la resistencia de un cocodrilo se sabe que su secuencia de aminoácidos $A$, debe contener como prefijo a $T$. Como en la BIOFAM hubo recortes de presupuesto solo cuentan con una máquina que puede hacer dos modificaciones genéticas:  
1. Extrae y elimina el primer aminoácido de $S$ y lo coloca al principio de $A$  
2. Extrae y elimina el primer aminoácido de $S$ y lo coloca al final de $A$  

Encuentre la **cantidad de secuencias de operaciones posibles** tales que la gallina genéticamente modificada sea resistente a las enfermedades.

## Soluciones
Para una primera solución se realizó un algoritmo en el que se hace un backtrack por cada una de las operaciones posibles para crear la cadena $S$ comprobando finalmente si contiene a la cadena $T$ como prefijo dando como resultado una secuencia posible de operaciones.

In [13]:
def count_backtrack(T:str, A:str):
    if 0 < len(T) and len(T) <= len(A):
        def backtrack(T:str, A:str, index:int, S:str):
            if len(A) == index:
                if T == S[0:len(T)]:return 1
                return 0
    
            char = A[index]
            cant = backtrack(T, A, index+1, char+S)
            cant += backtrack(T, A, index+1, S+char)
            return cant
        return backtrack(T, A, 0, "")
    return 0

Probando el algoritmo con distintos casos se obtiene como resultado:

In [14]:
print(count_backtrack('ACAC','ACACBCA'))
print(count_backtrack('ABAB','BABA'))
print(count_backtrack('ACABD','ACABDBCACA'))
print(count_backtrack('DBACA', 'ACABD'))
print(count_backtrack('AABBAA','BABAAA'))
print(count_backtrack('AABBAA','CHAABBAA'))
print(count_backtrack('ACAB','BDACACBA'))

12
4
20
4
12
4
8


Para el ejemplo donde $T$ (prefijo) es "ABAB" y la cadena $A$ es BABA se tiene:  
1. B $\rightarrow$ BA $\rightarrow$ BAB $\rightarrow$ BABA  (poniendo B al principio y al final)  
2. B $\rightarrow$ AB $\rightarrow$ BAB $\rightarrow$ BABA  (poniendo B al principio y al final)  
 
*Resultado*: 4

Como propuesta para una segunda solución se realizó un memoize al algoritmo anterior, ya que en algunos llamados se repetían calculos que ya se habían realizado con anterioridad, por lo que era mejor guardar por subsecuencias de la cadena formada $S$, la subcadena restante de $A$, así como la cantidad de subsecuencias de operaciones posibles realizadas

In [15]:
def count_backtrack_memoize(T:str, A:str):
    if 0 < len(T) and len(T) <= len(A):
        memoize = {}
        def backtrack(T:str, A:str, index:int, S:str):
            if len(A) == index:
                if T == S[0:len(T)]:return 1
                return 0
            char = A[index]
            dict_str = A[index+1:len(A)]
            word_form = char + S
            if dict_str in memoize and word_form in memoize[dict_str]: 
                cant = memoize[dict_str][word_form]
            else: 
                cant = backtrack(T, A, index+1, word_form)
                memoize[dict_str] = {word_form : cant}
            word_form = S + char
            if dict_str in memoize and word_form in memoize[dict_str]: 
                cant += memoize[dict_str][word_form]
            else:
                cant2 = backtrack(T, A, index+1, word_form)
                memoize[dict_str] = {word_form : cant2}
                cant += cant2
            return cant
        return backtrack(T, A, 0, "")
    return 0

Probando el algoritmo se puede comprobar que en efecto se obtienen los mismos resultados:

In [16]:
print(count_backtrack_memoize('ACAC','ACACBCA'))
print(count_backtrack_memoize('ABAB','BABA'))
print(count_backtrack_memoize('ACABD','ACABDBCACA'))
print(count_backtrack_memoize('DBACA', 'ACABD'))
print(count_backtrack_memoize('AABBAA','BABAAA'))
print(count_backtrack_memoize('AABBAA','CHAABBAA'))
print(count_backtrack_memoize('ACAB','BDACACBA'))

12
4
20
4
12
4
8


Se puede ver que aunque ambos algoritmos resuelven el problema, presentan una gran complejidad temporal ya que el backtrack (normal) y el backtrack con memoize (en el peor de los casos no se repiten patrones) pertencen al conjunto $O(2^n)$

Fue necesario encontrar otra vía para solucionar el ejercicio. Partiremos de los posibles casos en que puede aparecer la cadena $T$ (el prefijo), en la cadena creada (genéticamente modificada)

In [17]:
def count_naive_recursive(T:str, A:str):
    if 0 < len(T) and len(T) <= len(A):
        def naive_recursive(A,T):
            if len(A) < len(T): return 0
            if len(A)==1: 
                if len(T)==1 and T[0]==A[0]: return 2
                return 0
            if len(T)==0 : return 0
            output = naive_recursive(A[0:len(A)-1],T)
            if A[-1]==T[0]: 
                if len(T)==1: output += 2**(len(A)-1)
                else: output += naive_recursive(A[0:len(A)-1],T[1:])
            if len(A)==len(T) and A[-1]==T[-1]: output += naive_recursive(A[0:len(A)-1],T[0:len(T)-1])
            return output
        return naive_recursive(A,T)
    return 0

In [18]:
print(count_naive_recursive('ACAC','ACACBCA'))
print(count_naive_recursive('ABAB','BABA'))
print(count_naive_recursive('ACABD','ACABDBCACA'))
print(count_naive_recursive('DBACA', 'ACABD'))
print(count_naive_recursive('AABBAA','BABAAA'))
print(count_naive_recursive('AABBAA','CHAABBAA'))
print(count_naive_recursive('ACAB','BDACACBA'))

12
4
20
4
12
4
8


In [19]:
def count_dynamic(T:str, A:str):
    if len(T) > len(A): return 0
    dp=[ [ [0 for i in range(len(A))] for i in range(len(T))] for i in range(len(T)) ]
    i=len(T)-1
    while i>=0:
        for j in range(i,len(T)):
            for k in range(len(A)):
                m=j-i
                if m > k:
                    dp[i][j][k]=0
                    continue
                if k==0:
                    if A[0] == T[i]: dp[i][j][k]=2
                    else: dp[i][j][k]=0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
                    continue
                if k >0 : dp[i][j][k]=dp[i][j][k-1]
                if A[k] == T[i]:
                    if m==0: dp[i][j][k]+=2**k
                    elif i<len(T)-1 and k>0: dp[i][j][k]+=dp[i+1][j][k-1]
                if k==m and A[k] == T[j] and j>0 and k>0: dp[i][j][k]+=dp[i][j-1][k-1]
        i-=1
    return dp[0][len(T)-1][len(A)-1]

In [20]:
print(count_dynamic('ACAC','ACACBCA'))
print(count_dynamic('ABAB','BABA'))
print(count_dynamic('ACABD','ACABDBCACA'))
print(count_dynamic('DBACA', 'ACABD'))
print(count_dynamic('AABBAA','BABAAA'))
print(count_dynamic('AABBAA','CHAABBAA'))
print(count_dynamic('ACAB','BDACACBA'))

12
4
20
4
12
4
8


En las siguientes líneas de código se realizó un generador de cadenas que juegan el rol de prefijo (cocodrilo) y genoma a modificar (gallina) haciendo particiones en el prefijo, insertando un caracter random en alguna partición e invirtiendo la secuencia. Se comprueban los resultados entre los algoritmos.

In [21]:
import random
VOC = ["C","T","G","A","U","R","D"]

def generate_prefix(a=10,b=15):
    length = random.randint(a, b)
    prefix = ""
    while length > 0:
        letter = random.randint(0, len(VOC)-1)
        pos = random.randint(0,1)
        if pos:
            prefix += VOC[letter]
        else:
            prefix = VOC[letter] + prefix
        length-=1
    return prefix

def generate_secuence(prefix:str):
    secuence = ""
    while len(secuence) < 45:
        letter = random.randint(0, len(VOC)-1)
        partition = list(prefix.partition(random.choice(prefix)))
        pos = random.randint(0,len(partition))
        partition.insert(pos, VOC[letter])
        secuence += "".join(reversed(partition))
        secuence
    return secuence
   
TEST = 100
while TEST > 0:
    prefix = generate_prefix()
    secuence = generate_secuence(prefix)
    #c_b = count_backtrack(prefix,secuence)
    #c_b_m = count_backtrack_memoize(prefix,secuence)
    c_n_r = count_naive_recursive(prefix,secuence)
    print(len(secuence))
    print(c_n_r)
    c_d = count_dynamic(prefix,secuence)
    #print(c_b, c_b_m, c_n_r, c_d)
    print(c_d)
    print()
    #assert c_b == c_b_m and c_b_m == c_n_r == c_d
    assert c_n_r == c_d
    TEST-=1

56
120
120

48
22050582
22050582

56
476824
476824

52
131577552
131577552

48
532
532

56
61728
61728

48
3460
3460

52
44066
44066

55
476
476

56
4266020
4266020

52
10124
10124

52
18488
18488

48
22
22

48
5392
5392

52
126573516
126573516

48
12
12

48
250
250

52
1484
1484

55
143216868
143216868

45
0
0

45
0
0

48
24
24

45
580
580

56
0
0

52
730
730

52
538
538

45
2502
2502

45
0
0

48
13714900
13714900

55
17072128
17072128

48
28
28

52
572371132
572371132

48
24
24

48
850
850

48
58
58

55
1204
1204

56
21312
21312

52
488
488

48
1114312
1114312

55
8276
8276

52
315678
315678

48
1132
1132

48
700
700

56
1786
1786

55
346852
346852

48
3196
3196

48
44
44

55
957542528
957542528

52
14294228
14294228

48
28
28

45
1124
1124

55
37907846
37907846

48
76
76

52
440
440

56
44
44

52
0
0

52
160
160

52
26
26

56
14
14

48
22
22

56
274
274

52
20
20

56
1052
1052

55
428
428

52
15828
15828

48
328
328

52
1638015896
1638015896

52
48
48

45
32
32

52
4700260
4700260

